# Dependencies
### Importing Dependencies and using Splinter

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser

In [3]:
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

# NASA Mars News
### Scraping featured article and text from news site

In [4]:
news_paragraph_url = "https://mars.nasa.gov/news/"
browser.visit(news_paragraph_url)

news_paragraph_html = browser.html
news_paragraph_soup = bs(news_paragraph_html, "html.parser")

news_title = news_paragraph_soup.find("div", class_="content_title").find("a").text
news_p = news_paragraph_soup.find("div", class_="article_teaser_body").text

print(news_title)
print(news_p)

Six Things to Know About NASA's Opportunity Rover
Opportunity's mission is complete. Here are highlights from its time on Mars.


# JPL Mars Space Images
### Scraping featured image

In [8]:
space_images_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(space_images_url)

space_images_html = browser.html
featured_image_soup = bs(space_images_html, "html.parser")

featured_image_link = featured_image_soup.find("article")["style"].replace("background-image: url('", "").replace("');", "")
print(featured_image_link)

web_link = "https://www.jpl.nasa.gov"
featured_image_url = web_link + featured_image_link
print(featured_image_url)

/spaceimages/images/wallpaper/PIA18907-1920x1200.jpg
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18907-1920x1200.jpg


# Mars Weather
### Scraping latest tweet

In [29]:
mars_weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(mars_weather_url)

mars_weather_html = browser.html
mars_weather_soup = bs(mars_weather_html, "html.parser")

mars_weather_tweets = mars_weather_soup.find_all("div", class_="js-tweet-text-container")
for each_tweet in mars_weather_tweets:
    tweet_text = each_tweet.find("p").text
    if "pic.twitter.com" not in tweet_text:
        mars_weather = each_tweet.find("p").text
        print(mars_weather)
        break

Sol 2316 (2019-02-10), high -14C/6F, low -72C/-97F, pressure at 8.14 hPa, daylight 06:47-18:52


# Mars Facts
### Scraping data as Pandas dataframe

In [32]:
mars_facts_url = "http://space-facts.com/mars/"

mars_facts_df = pd.read_html(mars_facts_url)[0]
mars_facts_df.columns=["description", "value"]
mars_facts_df.set_index("description", inplace=True)
mars_facts_df

mars_facts_html = mars_facts_df.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

# Mars Hemispheres
### Scraping hemisphere images

In [42]:
mars_hemispheres_link = "https://astrogeology.usgs.gov"
mars_hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hemispheres_url)

mars_hemispheres_html = browser.html
mars_hemispheres_soup = bs(mars_hemispheres_html, "html.parser")

hemisphere_image_urls = []

mars_hemispheres_list = mars_hemispheres_soup.find_all("div", class_="item")

for each_hemisphere in mars_hemispheres_list:
    title = each_hemisphere.find("h3").text
    
    mars_hemispheres_image_link = each_hemisphere.find("a", class_="itemLink product-item")["href"]
    mars_hemispheres_download_url = mars_hemispheres_link + mars_hemispheres_image_link
    
    browser.visit(mars_hemispheres_download_url)
    mars_hemispheres_download_html = browser.html
    mars_hemispheres_download_soup = bs(mars_hemispheres_download_html, "html.parser")
    
    mars_hemispheres_full_image_link = mars_hemispheres_download_soup.find("img", class_="wide-image")["src"]
    mars_hemispheres_image_url = mars_hemispheres_link + mars_hemispheres_full_image_link

    hemisphere_image_urls.append({"title" : title, "img_url" : mars_hemispheres_image_url})
    
for each_image_url in hemisphere_image_urls:
    print(each_image_url)

{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}
{'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}
{'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}
{'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}


### Quitting splinter

In [44]:
browser.quit()